# tensordoku

Here, we will play with the idea of using tensors to capture the logic of sudoku puzzles.

The goal is to encode as little of the process of solving a sudoku in coded logic as possible.

Aside: The logic is of course still there, but it'll just be in tensor form!


In [145]:
import numpy as np
import math

## Sudoku puzzles

A sudoku puzzle consists of a grid, wherein each cell contains a single digit satisfying several rules.

For a puzzle of order $K \in N$, these rules are:
* The digits in the puzzle are $D = \lbrace d \in N | d < K^2 + 1\rbrace$, i.e. $D = \lbrace1, 2, ..., K^2\rbrace$
* Each row contains **one** of each of the digits in $D$
* Each column contains **one** of each of the digits in $D$
* Each non-overlapping $K \times K$ subgrid contains **one** of each of the digits in $D$

### Examples

The $K = 1$ case is trivial:
```
1
```

The $K = 2$ case is where it begins to be interesting:
```
1 2 3 4
3 4 1 2
2 3 4 1
4 1 2 3
```

and so on...

### Enter the Tensor

We can conveniently represent the state of a puzzle as a tensor.

Specifically, $T$ is a $K^2 \times K^2 \times K^2$ binary tensor representing the state of the puzzle, where:
* $T_{r,c,k} = \lbrace 1 \text{ if row } r, \text{ column } c \text{ could contain the digit } k \text{; } 0 \text{ otherwise} \rbrace$

A completely blank puzzle would be a $K^2$ length hyper-cube solely containing 1's, while a fully solved puzzle would contain values satisfying a family of constraint functions of the form $g_X(T) = 1$. The constraints are as follows:

#### One Digit Per Axis of Reduction

Each row must contain one of each of the digits. Thus when you sum-reduce $T$ over columns:

$g_C(T) = \sum_{c}T_{r,c,k} = 1;  \forall \lbrace r, k \rbrace$

Each column must contain one of each of the digits. Thus when you sum-reduce $T$ over rows:

$g_R(T) = \sum_{r}T_{r,c,k} = 1;  \forall \lbrace c, k \rbrace$

Each cell can contain only one of the digits. Thus when you sum-reduce $T$ over digits (depth):

$g_D(T) = \sum_{k}T_{r,c,k} = 1;  \forall \lbrace r, c \rbrace$

Conveniently, each of these three constraints have identical forms as single-axis sums across each of the respective axes. This results in each constraint producing a 2-D array over the non-reduced axes.

#### One Digit Per Subsection

The final constraint type is that the puzzle is divided into $K^2$ subsections $B_i$, wherein each digit can only appear once. Typically, these subsections are $K \times K$ blocks, which themselves are arranged in a $K \times K$ grid, giving the final $K^2 \times K^2$ array dimensions. Other variations exist. "Snakes on a Sudoku" was particularly entertaining.

$g_B(T) = \sum_{\lbrace r,c \rbrace \in B_i} T_{r,c,k} = 1; \forall \lbrace B_i, k \rbrace $

Alternatively, the subsection structure can be captured via masks in an accompanying $K^2 \times K^2 \times K^2$ tensor $B$, with the first two axes capturing the row and column and the final axis captuing which block that corresponding cell belongs to.

$g_B(T) = \sum_{r,c}B_{r,c,i}*T_{r,c,k} = 1; \forall \lbrace i, k \rbrace $

where $*$ is element-wise multiplication

## Class Design

The puzzle tensor $T$ makes sense to implement as a class. It has an easy encapsulation of data and methods that are only useful as a unit. Specifically:

* Construction from a 2-D human readable puzzle array and conversion back
* The $K^2 \times K^2 \times K^2$ tensor $T$ is the primary data member
* The order $K$ for convenience
* The structure capturing the sub-block designations for the puzzle (this is static as all puzzles of the same order share this logic)
* The method for updating the tensor to account for a constrained cell and propagate its DoF reduction
* For good measure, we can cache the constrained cells that have already been propagated so as to not repeat their propagations. This can let us keep track of which solved cells are "new".
* TODO(msmart): We can also keep track of which constraints are "newly" = 1, thereby moving the caching upstream 1 layer

Additionally:
* The methods for evaluating the constraints could be free functions as they take a tensor as input and do not change it
* TODO(msmart): what is the "pythonic" way of doing free functions?


In [146]:
class SudokuTensor:
    
    def __init__(self, puzzle: np.array):
        assert(np.shape(puzzle)[0] == np.shape(puzzle)[1])

        side = np.shape(puzzle)[0]

        self.tensor = np.zeros([side, side, side], np.int32)
        for i in range(side):
            self.tensor[:,:,i] = (puzzle == i + 1) + (puzzle == 0)

        self.order = compute_sudoku_tensor_order(self.tensor)
        self.block_labels = create_block_labels(self.order)
        self.propagated_cells = np.zeros([side, side], np.bool)
        
    def fix_cell_and_propagate(self, row, col, digit) -> None:
        if self.propagated_cells[row, col]:
            # Already propagated
            return

        # Propagate/Substitute value fixed by constraint into other co-constrained cells
        ## Common Cell
        self.tensor[row, col, :] = 0
        ## Common Column
        self.tensor[row, :, digit] = 0
        ## Common Row
        self.tensor[:, col, digit] = 0
        ## Common Block / Subsection
        self.tensor[self.block_labels == self.block_labels[row, col], digit] = 0
        
        # Fix Target Cell
        self.tensor[row, col, digit] = 1
        self.propagated_cells[row, col] = True
        return
        
    def get_puzzle(self) -> np.array:
        puzzle = np.zeros([self.order**2, self.order**2], np.int32)
        
        # Solved cells have their unique digit deduced
        solved_cells = np.sum(self.tensor, 2) == 1
        for i in range(self.order**2):
            puzzle += solved_cells * self.tensor[:, :, i] * (i + 1)

        return puzzle            
        
def compute_sudoku_tensor_order(tensor: np.array) -> int:
    side = int(np.shape(tensor)[0])
    order = int(math.sqrt(side+0.01))
    assert(order*order == side)
    return order

def create_block_labels(order: int) -> np.array:
    block = np.zeros([order, order], np.int32)
    block_label = block
    for i_c in range(order - 1):
        block_label = np.concatenate((block_label, i_c + 1 + block), axis = 1)
        
    block_row = block_label
    for i_r in range(order - 1):
        block_label = np.concatenate((block_label, (i_r + 1)*order + block_row), axis = 0)

    return block_label

### A Test Sample

Let's start with a simple example:
```
1 2 3 4
3 4 1 2
2 3 4 1
4 1 2 3
```

where $K = 2$

In [147]:
sample_puzzle = np.array([
    [1, 2, 3, 4],
    [3, 4, 1, 2],
    [2, 3, 4, 1],
    [4, 1, 2, 3]], np.int32)

test_tensor = SudokuTensor(sample_puzzle)
print(test_tensor.get_puzzle())

[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [4 1 2 3]]


Puzzles with unknown cell entries can be represented as $0$ s

In [148]:
sample_puzzle2 = np.array([
    [0, 2, 3, 4],
    [3, 0, 1, 2],
    [2, 3, 0, 1],
    [0, 1, 2, 0]], np.int32)

print(SudokuTensor(sample_puzzle2).get_puzzle())

[[0 2 3 4]
 [3 0 1 2]
 [2 3 0 1]
 [0 1 2 0]]


## Constraint Coding

Each of the constraint functions $g_X(T)$ can also be placed in functions:

In [149]:
s_tensor2 = SudokuTensor(sample_puzzle2)

def compute_row_constraints(sudoku_tensor: SudokuTensor) -> np.array:
    return sudoku_tensor.tensor.sum(axis=0)

def compute_col_constraints(sudoku_tensor: SudokuTensor) -> np.array:
    return sudoku_tensor.tensor.sum(axis=1)

def compute_digit_constraints(sudoku_tensor: SudokuTensor) -> np.array:
    return sudoku_tensor.tensor.sum(axis=2)

def compute_block_constraints(sudoku_tensor: SudokuTensor) -> np.array:
    block_constraints = np.zeros([sudoku_tensor.order**2, sudoku_tensor.order**2], np.int32) # block, digit
    for i_b in range(sudoku_tensor.order**2):
        sub_block = sudoku_tensor.tensor[sudoku_tensor.block_labels == i_b, :]
        block_constraints[i_b, :] = sub_block.sum(axis=0)
        
    return block_constraints

print('Sample Puzzle: \n', sample_puzzle2)
print('Row-Reduction Constraints: \n', compute_row_constraints(SudokuTensor(sample_puzzle2)))
print('Col-Reduction Constraints: \n', compute_col_constraints(SudokuTensor(sample_puzzle2)))
print('Digit-Reduction Constraints: \n', compute_digit_constraints(SudokuTensor(sample_puzzle2)))
print('Block-Reduction Constraints: \n', compute_block_constraints(SudokuTensor(sample_puzzle2)))

Sample Puzzle: 
 [[0 2 3 4]
 [3 0 1 2]
 [2 3 0 1]
 [0 1 2 0]]
Row-Reduction Constraints: 
 [[2 3 3 2]
 [2 2 2 1]
 [2 2 2 1]
 [2 2 1 2]]
Col-Reduction Constraints: 
 [[1 2 2 2]
 [2 2 2 1]
 [2 2 2 1]
 [3 3 2 2]]
Digit-Reduction Constraints: 
 [[4 1 1 1]
 [1 4 1 1]
 [1 1 4 1]
 [4 1 1 4]]
Block-Reduction Constraints: 
 [[2 3 3 2]
 [1 1 1 1]
 [2 2 2 1]
 [3 3 2 2]]


A useful observation can be made: since the elements of the tensor represent **possible** values a cell could take, all of the constraints should be at least $1$ if it is possible for them to be satisfied. A value of $0$ would indicate that the constraint cannot be satisfied for the given puzzle.

This allows us to define functions to verify the validity of the puzzle. Similarly, we can also define a function to check if a puzzle is fully solved.

In [150]:
def print_tensor_constraints(sudoku_tensor: SudokuTensor) -> None:
    print('R Constraints: \n', compute_row_constraints(sudoku_tensor))
    print('C Constraints: \n', compute_col_constraints(sudoku_tensor))
    print('D Constraints: \n', compute_digit_constraints(sudoku_tensor))
    print('B Constraints: \n', compute_block_constraints(sudoku_tensor))

def check_tensor_validity(sudoku_tensor: SudokuTensor) -> bool:        
    row_check_fails = (compute_row_constraints(sudoku_tensor) == 0)
    col_check_fails = (compute_col_constraints(sudoku_tensor) == 0)
    digit_check_fails = (compute_digit_constraints(sudoku_tensor) == 0)
    impossible_constraints = np.sum(row_check_fails) + np.sum(col_check_fails) + np.sum(digit_check_fails)
    return impossible_constraints == 0

def check_sudoku_tensor_solved(sudoku_tensor: SudokuTensor) -> bool:
    if (np.any(compute_row_constraints(sudoku_tensor) != 1)
        or np.any(compute_col_constraints(sudoku_tensor) != 1)
        or np.any(compute_digit_constraints(sudoku_tensor) != 1)
        or np.any(compute_block_constraints(sudoku_tensor) != 1)):
            return False
    else:
        return True

def check_puzzle(puzzle: np.array) -> bool:
    return check_tensor_validity(SudokuTensor(puzzle))

sample_puzzle3 = np.array([
    [3, 2, 3, 4],
    [3, 1, 1, 2],
    [2, 3, 2, 1],
    [4, 1, 2, 4]], np.int32)

print(sample_puzzle3)
print('Valid: \n',check_puzzle(sample_puzzle3))
print_tensor_constraints(SudokuTensor(sample_puzzle3))

[[3 2 3 4]
 [3 1 1 2]
 [2 3 2 1]
 [4 1 2 4]]
Valid: 
 False
R Constraints: 
 [[0 1 2 1]
 [2 1 1 0]
 [1 2 1 0]
 [1 1 0 2]]
C Constraints: 
 [[0 1 2 1]
 [2 1 1 0]
 [1 2 1 0]
 [1 1 0 2]]
D Constraints: 
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
B Constraints: 
 [[1 1 2 0]
 [1 1 1 1]
 [1 1 1 1]
 [1 2 0 1]]


Second, any constraint entries that are equal to **exactly** $1$ have no excess degrees of freedom, i.e. they lead to a solution of the form $T_{r,c,k} = 1$ for a **specific** $\lbrace r, c, k \rbrace$ cell index triplet. This solved cell can be propagated across the tensor via the other constraints dependent on the specific $\lbrace r, c, k \rbrace$ triplet by setting the associated "adjacent" cell entries to zero. As this propagation reduces the degrees of freedom available for satisfying other constraints, we can call these restricting functions _constriction_ functions.

In [151]:
def digit_constriction(sudoku_tensor: SudokuTensor) -> bool:
    prior_tensor = sudoku_tensor.tensor.copy()

    digit_constraints = compute_digit_constraints(sudoku_tensor)
    fixed_digit_entries = np.transpose((digit_constraints == 1).nonzero())
    for fixed_digit in fixed_digit_entries:
        digit_idx = (sudoku_tensor.tensor[fixed_digit[0],fixed_digit[1],:] == 1).nonzero()
        # Propagate
        sudoku_tensor.fix_cell_and_propagate(fixed_digit[0], fixed_digit[1], digit_idx)

    return np.any(sudoku_tensor.tensor != prior_tensor)

def col_constriction(sudoku_tensor: SudokuTensor) -> bool:
    prior_tensor = sudoku_tensor.tensor.copy()

    col_constraints = compute_col_constraints(sudoku_tensor)
    fixed_col_entries = np.transpose((col_constraints == 1).nonzero())
    for fixed_col in fixed_col_entries:
        col_idx = (sudoku_tensor.tensor[fixed_col[0], :, fixed_col[1]] == 1).nonzero()
        # Propagate
        sudoku_tensor.fix_cell_and_propagate(fixed_col[0], col_idx, fixed_col[1])

    return np.any(sudoku_tensor.tensor != prior_tensor)

def row_constriction(sudoku_tensor: SudokuTensor) -> bool:
    prior_tensor = sudoku_tensor.tensor.copy()
    
    row_constraints = compute_row_constraints(sudoku_tensor)
    fixed_row_entries = np.transpose((row_constraints == 1).nonzero())
    for fixed_row in fixed_row_entries:
        row_idx = (sudoku_tensor.tensor[:, fixed_row[0], fixed_row[1]] == 1).nonzero()
        # Propagate
        sudoku_tensor.fix_cell_and_propagate(row_idx, fixed_row[0], fixed_row[1])
        
    return np.any(sudoku_tensor.tensor != prior_tensor)

def block_constriction(sudoku_tensor: SudokuTensor) -> bool:
    prior_tensor = sudoku_tensor.tensor.copy()

    block_constraints = compute_block_constraints(sudoku_tensor)
    fixed_block_entries = np.transpose((block_constraints == 1).nonzero())
    for fixed_block in fixed_block_entries:
        # fixed_block_entries, axis0 is block label, axis1 is digit
        # So need to map this pair to the corresponding row,col pair
        block_mask = sudoku_tensor.block_labels == fixed_block[0]
        digit_mask = sudoku_tensor.tensor[:, :, fixed_block[1]]
        block_digit_entry = np.transpose((block_mask * digit_mask).nonzero())[0] # [0] because MUST be unique (1 entry)
        # block_digit_entry [0] is now row, [1] is col
        # Propagate
        sudoku_tensor.fix_cell_and_propagate(block_digit_entry[0], block_digit_entry[1], fixed_block[1])
        
    return np.any(sudoku_tensor.tensor != prior_tensor)

Applying each of these constrictions affects the puzzle solution via different constraints:

In [152]:
print('Results of Successive digit_constriction()')
print('Puzzle: \n',sample_puzzle2)
sudoku_tensor3 = SudokuTensor(sample_puzzle2)
print(digit_constriction(sudoku_tensor3))
print(sudoku_tensor3.get_puzzle())
print(digit_constriction(sudoku_tensor3))
print(sudoku_tensor3.get_puzzle())
print(digit_constriction(sudoku_tensor3))
print(sudoku_tensor3.get_puzzle())

print('\nResults of Successive col_constriction()')
print('Puzzle: \n',sample_puzzle2)
sudoku_tensor4 = SudokuTensor(sample_puzzle2)
print(col_constriction(sudoku_tensor4))
print(sudoku_tensor4.get_puzzle())
print(col_constriction(sudoku_tensor4))
print(sudoku_tensor4.get_puzzle())
print(col_constriction(sudoku_tensor4))
print(sudoku_tensor4.get_puzzle())

print('\nResults of Successive row_constriction()')
print('Puzzle: \n',sample_puzzle2)
sudoku_tensor5 = SudokuTensor(sample_puzzle2)
print(row_constriction(sudoku_tensor5))
print(sudoku_tensor5.get_puzzle())
print(row_constriction(sudoku_tensor5))
print(sudoku_tensor5.get_puzzle())
print(row_constriction(sudoku_tensor5))
print(sudoku_tensor5.get_puzzle())

print('\nResults of Successive block_constriction()')
print('Puzzle: \n',sample_puzzle2)
sudoku_tensor6 = SudokuTensor(sample_puzzle2)
print(block_constriction(sudoku_tensor6))
print(sudoku_tensor6.get_puzzle())
print(block_constriction(sudoku_tensor6))
print(sudoku_tensor6.get_puzzle())
print(block_constriction(sudoku_tensor6))
print(sudoku_tensor6.get_puzzle())

Results of Successive digit_constriction()
Puzzle: 
 [[0 2 3 4]
 [3 0 1 2]
 [2 3 0 1]
 [0 1 2 0]]
True
[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [4 1 2 3]]
False
[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [4 1 2 3]]
False
[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [4 1 2 3]]

Results of Successive col_constriction()
Puzzle: 
 [[0 2 3 4]
 [3 0 1 2]
 [2 3 0 1]
 [0 1 2 0]]
True
[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [0 1 2 0]]
True
[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [4 1 2 3]]
False
[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [4 1 2 3]]

Results of Successive row_constriction()
Puzzle: 
 [[0 2 3 4]
 [3 0 1 2]
 [2 3 0 1]
 [0 1 2 0]]
True
[[0 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [0 1 2 3]]
True
[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [4 1 2 3]]
False
[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [4 1 2 3]]

Results of Successive block_constriction()
Puzzle: 
 [[0 2 3 4]
 [3 0 1 2]
 [2 3 0 1]
 [0 1 2 0]]
True
[[0 2 3 4]
 [3 0 1 2]
 [2 3 0 1]
 [4 1 2 0]]
True
[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [4 1 2 3]]
False
[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [4 1 2 3]]


We can watch the constraints change as we iteratively use the exactly satisfied constraints ($1$ values) to successively constrict the puzzle:

In [153]:
print('Sample Puzzle: \n', sample_puzzle2)
sample_tensor = SudokuTensor(sample_puzzle2)
print_tensor_constraints(sample_tensor)
print('Apply Row Constriction')
print(row_constriction(sample_tensor))
print_tensor_constraints(sample_tensor)
print('Apply Col Constriction')
print(col_constriction(sample_tensor))
print_tensor_constraints(sample_tensor)
print('Apply Digit Constriction')
print(digit_constriction(sample_tensor))
print_tensor_constraints(sample_tensor)
print('Apply Block Constriction')
print(block_constriction(sample_tensor))
print_tensor_constraints(sample_tensor)

Sample Puzzle: 
 [[0 2 3 4]
 [3 0 1 2]
 [2 3 0 1]
 [0 1 2 0]]
R Constraints: 
 [[2 3 3 2]
 [2 2 2 1]
 [2 2 2 1]
 [2 2 1 2]]
C Constraints: 
 [[1 2 2 2]
 [2 2 2 1]
 [2 2 2 1]
 [3 3 2 2]]
D Constraints: 
 [[4 1 1 1]
 [1 4 1 1]
 [1 1 4 1]
 [4 1 1 4]]
B Constraints: 
 [[2 3 3 2]
 [1 1 1 1]
 [2 2 2 1]
 [3 3 2 2]]
Apply Row Constriction
True
R Constraints: 
 [[2 3 2 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
C Constraints: 
 [[1 2 2 1]
 [1 1 1 1]
 [1 1 1 1]
 [2 2 1 1]]
D Constraints: 
 [[3 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [3 1 1 1]]
B Constraints: 
 [[1 2 2 1]
 [1 1 1 1]
 [2 2 1 1]
 [1 1 1 1]]
Apply Col Constriction
True
R Constraints: 
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
C Constraints: 
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
D Constraints: 
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
B Constraints: 
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
Apply Digit Constriction
False
R Constraints: 
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
C Constraints: 
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 

As we can see, we can iteratively apply the constrictions until we reach a solution (or a steady state for our first-order solution).

In [154]:
def first_order_solution(sudoku_tensor: SudokuTensor, max_iter: int = 100) -> bool:
    iter = 0
    while iter < max_iter:
        iter += 1
        # Monitor for validity
        if not check_tensor_validity(sudoku_tensor):
            print('Infeasible solution!')
            return False # Give up due to infeasibility

        # Try digit constriction, then col constriction, then row constriction, then give up
        # This is a heuristic: in my experience, the digit constraint is dominant for most sudoku puzzles.
        # For harder puzzles, a more principled selection may be better
        # NOTE: foo_constriction -> bool [false: constriction changed nothing, true: else]
        if digit_constriction(sudoku_tensor):
            print('Digit Constriction changed something on iteration', iter)
            continue
        elif block_constriction(sudoku_tensor):
            print('Block Constriction changed something on iteration', iter)
            continue
        elif col_constriction(sudoku_tensor):
            print('Col Constriction changed something on iteration', iter)
            continue
        elif row_constriction(sudoku_tensor):
            print('Row Constriction changed something on iteration', iter)
            continue
        else:
            if check_sudoku_tensor_solved(sudoku_tensor):
                print('Solved after ', iter, ' iterations')
                return True
            else:
                print('Stuck after ', iter, ' iterations')
                return False
    
    print('max_iter reached... Giving up...')
    return False # Give up due to iteration excess

In [155]:
sample_tensor = SudokuTensor(sample_puzzle)
print(first_order_solution(sample_tensor))
print(sample_tensor.get_puzzle())

Solved after  1  iterations
True
[[1 2 3 4]
 [3 4 1 2]
 [2 3 4 1]
 [4 1 2 3]]


Now let's actually test this out by finding some REAL puzzles (and by increasing K to 3 in the process):



In [156]:
real_sample1 = np.array([
    [6, 1, 3, 0, 0, 0, 0, 0, 9],
    [0, 0, 7, 0, 9, 0, 0, 3, 0],
    [0, 0, 0, 5, 0, 0, 6, 7, 0],
    [0, 0, 8, 9, 2, 6, 4, 1, 0],
    [0, 0, 9, 0, 0, 8, 7, 6, 3],
    [5, 0, 1, 3, 4, 7, 0, 2, 8],
    [0, 0, 0, 0, 6, 0, 1, 0, 7],
    [7, 0, 5, 0, 0, 0, 0, 0, 0],
    [0, 9, 6, 7, 0, 5, 0, 0, 4]], np.int32)

real_sudoku_tensor = SudokuTensor(real_sample1)
first_order_solution(real_sudoku_tensor)
print('Solution: \n',real_sudoku_tensor.get_puzzle())

Digit Constriction changed something on iteration 1
Digit Constriction changed something on iteration 2
Digit Constriction changed something on iteration 3
Digit Constriction changed something on iteration 4
Digit Constriction changed something on iteration 5
Digit Constriction changed something on iteration 6
Digit Constriction changed something on iteration 7
Digit Constriction changed something on iteration 8
Block Constriction changed something on iteration 9
Digit Constriction changed something on iteration 10
Digit Constriction changed something on iteration 11
Digit Constriction changed something on iteration 12
Digit Constriction changed something on iteration 13
Digit Constriction changed something on iteration 14
Digit Constriction changed something on iteration 15
Solved after  16  iterations
Solution: 
 [[6 1 3 8 7 2 5 4 9]
 [2 5 7 6 9 4 8 3 1]
 [9 8 4 5 1 3 6 7 2]
 [3 7 8 9 2 6 4 1 5]
 [4 2 9 1 5 8 7 6 3]
 [5 6 1 3 4 7 9 2 8]
 [8 3 2 4 6 9 1 5 7]
 [7 4 5 2 8 1 3 9 6]
 [1 9

That one worked! But it was easy... Let's try a harder one and see how it goes...

In [157]:
medium_sample = np.array([
    [5, 0, 0, 0, 0, 9, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 9],
    [0, 0, 0, 4, 5, 2, 0, 0, 6],
    [0, 8, 0, 9, 7, 4, 0, 0, 0],
    [2, 1, 0, 0, 3, 0, 4, 6, 0],
    [0, 0, 4, 2, 0, 1, 8, 0, 5],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [8, 0, 0, 0, 0, 7, 0, 0, 0],
    [0, 7, 3, 0, 9, 0, 6, 5, 0]], np.int32)

medium_sudoku_tensor = SudokuTensor(medium_sample)
first_order_solution(medium_sudoku_tensor)
print('Solution: \n',medium_sudoku_tensor.get_puzzle())

Digit Constriction changed something on iteration 1
Digit Constriction changed something on iteration 2
Digit Constriction changed something on iteration 3
Digit Constriction changed something on iteration 4
Digit Constriction changed something on iteration 5
Digit Constriction changed something on iteration 6
Digit Constriction changed something on iteration 7
Digit Constriction changed something on iteration 8
Digit Constriction changed something on iteration 9
Digit Constriction changed something on iteration 10
Digit Constriction changed something on iteration 11
Solved after  12  iterations
Solution: 
 [[5 6 8 3 1 9 7 2 4]
 [3 4 2 7 8 6 5 1 9]
 [1 9 7 4 5 2 3 8 6]
 [6 8 5 9 7 4 2 3 1]
 [2 1 9 8 3 5 4 6 7]
 [7 3 4 2 6 1 8 9 5]
 [9 2 6 5 4 3 1 7 8]
 [8 5 1 6 2 7 9 4 3]
 [4 7 3 1 9 8 6 5 2]]


Apparently that one was even easier for the solver? Let's try a hard-level from sudoku.com

In [158]:
hard_sample = np.array([
    [0, 3, 0, 6, 0, 2, 9, 0, 0],
    [0, 0, 0, 5, 0, 9, 0, 0, 0],
    [0, 4, 9, 0, 3, 0, 2, 0, 0],
    [0, 9, 0, 0, 0, 7, 0, 3, 0],
    [2, 1, 8, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 9, 6],
    [0, 0, 2, 0, 1, 5, 0, 0, 8],
    [1, 0, 0, 0, 0, 0, 4, 2, 0],
    [0, 7, 0, 0, 2, 0, 0, 6, 0]], np.int32)

blank_sample = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]], np.int32)

hard_sudoku_tensor = SudokuTensor(hard_sample)
first_order_solution(hard_sudoku_tensor)
print('Solution: \n',hard_sudoku_tensor.get_puzzle())

Digit Constriction changed something on iteration 1
Digit Constriction changed something on iteration 2
Digit Constriction changed something on iteration 3
Block Constriction changed something on iteration 4
Block Constriction changed something on iteration 5
Digit Constriction changed something on iteration 6
Digit Constriction changed something on iteration 7
Digit Constriction changed something on iteration 8
Block Constriction changed something on iteration 9
Stuck after  10  iterations
Solution: 
 [[0 3 0 6 0 2 9 0 0]
 [0 2 0 5 0 9 6 0 3]
 [6 4 9 0 3 0 2 0 0]
 [4 9 6 0 0 7 0 3 2]
 [2 1 8 0 0 0 0 0 0]
 [0 5 0 2 0 0 0 9 6]
 [9 6 2 4 1 5 3 7 8]
 [1 8 0 0 0 0 4 2 0]
 [0 7 4 0 2 0 0 6 0]]
